# Dive !

## Part 1 problem statement

(Adapted from [Advent of Code 2021, day 2](https://adventofcode.com/2021/day/2))

You will be given a series of instructions like

```txt
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

These instructions will change your horizontal position and your depth, two values you need to keep track of:

 - `forward X` increases the horizontal position by X units;
 - `down X` increases the depth by X units; and
 - `up X` decreases the depth by X units.

Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

 - `forward 5` adds 5 to your horizontal position, a total of 5.
 - `down 5` adds 5 to your depth, resulting in a value of 5.
 - `forward 8` adds 8 to your horizontal position, a total of 13.
 - `up 3` decreases your depth by 3, resulting in a value of 2.
 - `down 8` adds 8 to your depth, resulting in a value of 10.
 - `forward 2` adds 2 to your horizontal position, a total of 15.

After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

**Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?**

_Using the input file `input.txt`, the result should be 1727835._

In [1]:
INPUT_FILE = 'input.txt'

In [2]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth = 0, 0
for line in instructions:
    command, value = line.split()
    value = int(value)

    if command == "forward":
        horiz_pos += value
    elif command == "up":
        depth -= value
    elif command == "down":
        depth += value
    else:
        raise ValueError('Unknown command.')

print(horiz_pos * depth)
    


1727835


There is nothing too wild going on here.

Perhaps the thing that can easily go most unnoticed is the fact that the line `command, value = line.split()` is already doing some input validation for us:
the fact that we are unpacking into `command, line` means we are assuming that `line.split()` returns two values.
If it returns any other number of values, we get a `ValueError`:

In [3]:
command, value = "cmd val otherthing".split()

ValueError: too many values to unpack (expected 2)

### Pattern matching

If you are using Python 3.10 or newer, you might be tempted to use [structural pattern matching](https://mathspp.com/blog/pydonts/structural-pattern-matching-tutorial) here.
We can write a solution using `match` that is remarkably similar to the solution using `if`:

In [ ]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth = 0, 0
for line in instructions:
    command, value = line.split()
    value = int(value)
 
    match command:
        case "forward":
            horiz_pos += value
        case "up":
            depth -= value
        case "down":
            depth += value
        case _:
            raise ValueError("Unknown Command")


print(horiz_pos * depth)

1727835


So, is this any better?
We can argue it is _not_, because we didn't simplify our code, and yet managed to increase its depth.

To walk towards a scenario where pattern matching would be probably be more useful, let's rewrite the `match` statement:

In [ ]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth = 0, 0
for line in instructions:

    match line.split():
        case ["forward", value]:
            horiz_pos += int(value)
        case ["up", value]:
            depth -= int(value)
        case ["down", value]:
            depth += int(value)
        case _:
            raise ValueError("Unknown Command.")

print(horiz_pos * depth)


1727835


By matching directly the `line.split()` expression, we are making it easier for ourselves to handle instructions that have a different _structure_.
For example, imagine there was a `"reset"` instruction, that resetted the horizontal position and the depth to 0.
Using `match`, this is what the solution could look like:

In [ ]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

instructions.append("reset")
horiz_pos, depth = 0, 0
for line in instructions:
    match line.split():
        case ["reset"]:
            horiz_pos, depth = 0, 0
        case ["forward", value]:
            horiz_pos += int(value)
        case["up", value]:
            depth -= int(value)
        case["down", value]:
            depth += int(value)
        case _:
            raise ValueError("Unkown Command.")
print(horiz_pos * depth)

0


We only needed to add two lines of code to handle this new command, and the handling of all commands looks similar: a `case` statement and some code.
If we were to do the same thing in the original `if` statement, we would have to special-case the `"reset"` command because we would have to check for it before unpacking the line into the `command` and `line` variables:

In [ ]:
with open(INPUT_FILE, "r") as f:
    instructions = f.readlines()
    
instructions.append("reset")  # Add a "reset" command to the end.

horiz_pos, depth = 0, 0
for line in instructions:
    if line == "reset":
        horiz_pos, depth = 0, 0
        continue
    
    command, value = line.split()
    value = int(value)
    
    if command == "forward":
        horiz_pos += value
    elif command == "up":
        depth -= value
    elif command == "down":
        depth += value
    else:
        raise ValueError("Unknown command.")

print(horiz_pos * depth)  # Prints 0 because the last command was "reset".

0


So, in conclusion, for such a homogeneous set of commands, the `if` statement is preferable.
If the line structure were more heterogeneous, then structural pattern matching would start to show its benefits.

### How to end the `if` block

In the solution above, our `if` block compares `command` explicitly to each of the three possible commands, and uses the `else` to raise an error in the event that we receive a command we don't know.
We could have written, just as easily, the following `if` block:

```py
if command == "forward":
    horiz_pos += value
elif command == "up":
    depth -= value
else:
    depth += value
```

This block assumes that the variable `command` _always_ contains one of the three known commands, and thus uses the `else` to handle the `down` command.

However, there is a disadvantage to writing code like this:
one cannot look at the `if` block and _read_ what is the third case.
Is it a single one?
Are there multiple commands that map to the action of doing `depth += value`?

Thus, one can argue it is preferable to be explicit about the cases we are handling.
Of course, we can still choose to write the `if` block like so:

```py
if command == "forward":
    horiz_pos += value
elif command == "up":
    depth -= value
elif command == "down":
    depth += value
```

The difference, here, is that we do not include the `else` branch with the `raise` statement.
This says explicitly the commands that we are handling, while also showing that we do not expect to have to handle anything else.

Another slight variant would be to write

```py
if command == "forward":
    horiz_pos += value
elif command == "up":
    depth -= value
elif command == "down":
    depth += value
else:
    pass
```

This variant can be understood to mean “we assume something else might come through in the variable `command`, but we don't care about it”.

These are just minor variations of one another, and _your_ interpretation might not necessarily align with mine, but I find it to be an interesting exercise to think about the different ways in which similar pieces of code are read and understood.

---

As far as this problem is concerned, there isn't much we can do to improve our solution significantly.
The problem is straightforward enough that any attempts to be clever would do more harm than good.

Therefore, we will now cover the second part of the problem.
Then, because this is a fairly simple problem, it acts as a good toy example to introduce a couple of interesting tools that could be relevant for similar tasks, but that would represent too much overhead here.

## Part 2 problem statement

(Adapted from [Advent of Code 2021, day 2](https://adventofcode.com/2021/day/2))

Part 2 is a modification of part 1.
Now, not only do we have to keep track of the horizontal position and depth, we also have to keep track of the **aim**.
On top of that, the **same commands** now have a **different meaning**:


 - `down X` increases your aim by X units;
 - `up X` decreases your aim by X units; and
 - `forward X` does two things:
    - it increases your horizontal position by X units; and
    - it increases your depth by your aim multiplied by X.
    
Recall the previous example:

```txt
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

Now, the final result is different:

 - `forward 5` adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
 - `down 5` adds 5 to your aim, resulting in a value of 5.
 - `forward 8` adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
 - `up 3` decreases your aim by 3, resulting in a value of 2.
 - `down 8` adds 8 to your aim, resulting in a value of 10.
 - `forward 2` adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.

After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, **calculate the horizontal position and depth** you would have after following the planned course.
**What do you get if you multiply your final horizontal position by your final depth?**

_Using the input file `input.txt`, the answer should be 1544000595._

### Modifying the baseline solution

In order to solve this new version of the problem, we just have to adapt the handling of each command:

In [4]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth, aim = 0, 0, 0
for line in instructions:
    command, value = line.split()
    value = int(value)

    if command == "forward":
        horiz_pos += value
        depth += aim * value
    
    elif command == "down":
        aim += value
    
    elif command == "up":
        aim -= value

    else:
        raise ValueError("Commnad not found.")

print(horiz_pos * depth)

1544000595


### Rudimentary space-time complexity analysis

#### Time

Let us analyse the the space and time complexities of our solution, as a function of the number `n` of instructions.

A rule of thumb to estimate the time complexity of an algorithm is to sum the time complexities of things that happen after each other, and to multiply the time complexities of loops with the time complexities of the code inside them.

In our particular example, we have an outer `for` loop that goes through all instructions once, so that loop by itself is linear, or $O(n)$.
Now, we need to check the time complexity of the loop body, because the loop body gets executed in _each_ iteration.

As we can see, all operations inside the loop body execute in constant time: they do not depend on the total amount of instructions.
Hence, the loop body, for each iteration, is $O(1)$.

Putting it all together (in a not-so-rigorous manner), we get that the whole algorithm is $O(n) \times O(1) = O(n)$.

This shouldn't be surprising, and it is impossible to improve: we cannot know what the final horizontal position/depth/aim is without reading all instructions, and to read all instructions we need to go through the whole set of instructions at least once, which is already $O(n)$ by itself.

#### Space

The space complexity of our current solution is also linear, because we store all the instructions in a list.
We can reduce the space complexity to be constant if we employ the strategy of lazily iterating over the input file:

In [5]:
horiz_pos, depth, aim = 0, 0, 0

with open(INPUT_FILE, 'r') as f:
    for line in f:
        command, value = line.split()
        value = int(value)

        if command == "forward":
            horiz_pos += value
            depth += aim * value

        elif command == "up":
            aim -= value
        
        elif command == "down":
            aim += value
        
        else:
            raise ValueError("Command not found")
        
print(horiz_pos * depth)

1544000595


The space complexity of the modified code is $O(1)$ because we only store three integers.

## Other thoughts

As mentioned previously, let us use this toy problem as an excuse to cover a couple of other tools that you could benefit from.

### Parsing input

People have different sensibilities, so you may not relate to what I am about to say, but there is one small thing that annoys me a little bit in the solution above, and that is the parsing of each line.

We know that each line has a very nice format, but we still need to break it into pieces and do some conversions here and there.
A very reasonable thing to do would be to create an auxiliary function whose only job is to parse a line of input into its appropriate pieces.
For our challenge, we might even assume that the line _will_ have the appropriate format:

In [9]:
def parse_instruction_line(line):
    command, value = line.split()
    return command, int(value)

horiz_pos, depth, aim = 0, 0, 0

with open(INPUT_FILE, 'r') as f:
    for line in f:
        command, value = parse_instruction_line(line)
        
        if command == "forward":
            horiz_pos += value
            depth += aim * value

        elif command == "up":
            aim -= value
        
        elif command == "down":
            aim += value
        
        else:
            raise ValueError("Command not found")
        
print(horiz_pos * depth)

1544000595


For our little problem, it might not look very advantageous to define an auxiliary function to do that work.
However, as problems become more complex and as input formats become more complex/less structured, input parsing becomes a significant endeavour.
When that is the case, it is generally advised that you _separate concerns_:
have a function to do input parsing and then another function to do the number crunching/problem-solving.

### Enumerations of constants

Another tool that is quite useful comes from the `enum` module.
`enum` is short for “enumeration”, and is useful when you have related constant variables that you would like to keep together.

In our example, those (three) constants are the string values of the three commands:

 - “forward”
 - “up”
 - “down”

With enumerations, we can group values that act as “global constants” and use them instead of the actual values:

In [16]:
from enum import Enum

class Command(Enum):
    FORWARD = "forward"
    UP = "up"
    DOWN = "down"

horiz_pos, depth, aim = 0, 0, 0

with open(INPUT_FILE, 'r') as f:
    for line in f:
        command, value = line.split()
        value = int(value)
        command = Command(command) # We say that `command` is a `Command`, ...

        # ... and we compare it to each possible command:
        if command == Command.FORWARD:
            horiz_pos += value
            depth += aim * value

        elif command == Command.UP:
            aim -= value

        elif command == Command.DOWN:
            aim += value

        else:
            ValueError("Unkwon Command")
        
print(horiz_pos * depth)

1544000595


There are many benefits to using enums in this type of situation.
One of the best ones, for this case, is that using `Command` to create the command will do data validation for us.
If you try to create a `Command` instance out of something that is not a valid command, the code will break:

In [17]:
Command("Invalid")

ValueError: 'Invalid' is not a valid Command

This is a good thing!
If one day, the input data changes and there is a new command, you'll hit this error that tells you that you must adapt your code to the new command.

You can read about `enum` [in the docs](https://docs.python.org/3/library/enum.html) and you can also read this to understand better [why you would use `enum` in the first place](https://mathspp.com/blog/why-use-enums).

### `StrEnum`

If you're using Python 3.11 or newer, you can use `enum.StrEnum` instead of `Enum`.
This is just a specialised version of `Enum` for string values.
That, together with `enum.auto`, allow us to define our enumeration class in a more robust way:

In [19]:
from enum import auto, StrEnum

# We define an Enum(eration) with the valid commands.
class Command(StrEnum):
    FORWARD = auto()
    UP = auto()
    DOWN = auto()

Using `enum.auto` means we are not at risk of making typos when defining the values, and it also menas that if we need to change the name of the command, the value will be computed _automatically_.

### Structural pattern matching and enums

If you're using enums, then structural pattern matching becomes a more attractive alternative for a simple reason.
When you define an enum, your IDE can look at it and see how many different values exist.
In our case, three:

 1. `Command.FORWARD`;
 2. `Command.UP`; and
 3. `Command.DOWN`.

So, if later on you use structural pattern matching to match against the command, _your IDE can warn you if you forget to match against one or more values_!

So, suppose you wrote something like the code below, where you forgot to match against the value `Command.DOWN`.
If that were the case, your IDE would warn you that you had forgotten about `Command.DOWN`:

In [20]:
from enum import auto, StrEnum

class Command(StrEnum):
    FORWARD = auto()
    UP = auto()
    DOWN = auto()

horiz_pos, depth, aim = 0, 0, 0

with open(INPUT_FILE, 'r') as f:
    for line in f:
        command, value = line.split()
        value = int(value)
        command = Command(command)

        match command:
            case command.FORWARD:
                horiz_pos += value
                depth += aim * value
            
            case command.UP:
                aim -= value

print(horiz_pos * depth)

-1910857875


## Reduce the inputs to a single output

Let me rewrite the standard solution, using an `if` statement, so that:

 - the input is parsed immediately; and
 - the position update is done inside an auxiliary function:

In [22]:
def state_update(state, instruction):
    horiz_pos, depth, aim = state
    command, value = instruction

    if command == "forward":
        horiz_pos += value
        depth += aim * value
    elif command == "up":
        aim -= value
    elif command == "down":
        aim += value
    else:
        raise ValueError("Unknown command.")
    
    return (horiz_pos, depth, aim)

with open(INPUT_FILE, 'r') as f:
    instructions = [
        (command, int(value)) for command, value in map(str.split, f)
    ]

state = (0, 0, 0)
for instruction in instructions:
    state = state_update(state, instruction)

print(state[0] * state[1])

1544000595


Notice that the only purpose of the `for` loop is to update the `state` with the new information that comes from the instructions.
We're going over the list of `instructions` and we're reducing it to a single value: the final state.
So, if all the other sections made sense so far, I invite you to study this solution:

In [23]:
from functools import reduce  # <--

def state_update(state, instruction):
    horiz_pos, depth, aim = state
    command, value = instruction

    if command == "forward":
        horiz_pos += value
        depth += aim * value
    elif command == "up":
        aim -= value
    elif command == "down":
        aim += value
    else:
        raise ValueError("Unknown command.")
    
    return (horiz_pos, depth, aim)


with open(INPUT_FILE, "r") as f:
    instructions = [
        (command, int(value)) for command, value in map(str.split, f)
    ]

final_state = reduce(state_update, instructions, (0, 0, 0))  # <--
print(final_state[0] * final_state[1])


1544000595


`functools.reduce` is a function that is underappreciated, and that's because of a beautiful paradox around it:

 - while you will _almost never_ need to use it (explicitly),
 - I'm sure you use it very frequently (implicitly)!

That's because `functools.reduce` is the “algorithm” behind built-ins like `sum`, `any`, `all`, `min`, `max`, `math.prod`, and others.
I invite you to [learn more about `functools.reduce`](https://mathspp.com/blog/pydonts/the-power-of-reduce) because it will help you understand relationships between many built-ins you probably didn't know were related.